# Personal Finance Management

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import matplotlib.pyplot as plt

def setup_database():
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS income (
                        id INTEGER PRIMARY KEY,
                        date TEXT,
                        source TEXT,
                        amount REAL
                      )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                        id INTEGER PRIMARY KEY,
                        date TEXT,
                        category TEXT,
                        amount REAL
                      )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS savings (
                        id INTEGER PRIMARY KEY,
                        date TEXT,
                        amount REAL
                      )''')
    conn.commit()
    conn.close()


def add_income(source, amount, date):
    try:
        amount = float(amount)
        conn = sqlite3.connect('finance.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO income (source, amount, date) VALUES (?, ?, ?)", (source, amount, date))
        conn.commit()
        conn.close()
        refresh_income_list()
    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid amount.")

def add_expense(category, amount, date):
    try:
        amount = float(amount)
        conn = sqlite3.connect('finance.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO expenses (category, amount, date) VALUES (?, ?, ?)", (category, amount, date))
        conn.commit()
        conn.close()
        refresh_expense_list()
    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid amount.")


def add_savings(amount, date):
    try:
        amount = float(amount)
        conn = sqlite3.connect('finance.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO savings (amount, date) VALUES (?, ?)", (amount, date))
        conn.commit()
        conn.close()
        refresh_savings_list()
    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid amount.")


def refresh_income_list():
    for i in income_tree.get_children():
        income_tree.delete(i)
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM income")
    rows = cursor.fetchall()
    for row in rows:
        income_tree.insert('', tk.END, values=row)
    conn.close()


def refresh_expense_list():
    for i in expense_tree.get_children():
        expense_tree.delete(i)
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM expenses")
    rows = cursor.fetchall()
    for row in rows:
        expense_tree.insert('', tk.END, values=row)
    conn.close()


def refresh_savings_list():
    for i in savings_tree.get_children():
        savings_tree.delete(i)
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM savings")
    rows = cursor.fetchall()
    for row in rows:
        savings_tree.insert('', tk.END, values=row)
    conn.close()


def plot_expense_pie_chart():
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute("SELECT category, SUM(amount) FROM expenses GROUP BY category")
    data = cursor.fetchall()
    conn.close()

    categories = [row[0] for row in data]
    amounts = [row[1] for row in data]

    if not categories or not amounts:
        messagebox.showinfo("No Data", "No expense data available to plot.")
        return

    plt.figure(figsize=(6, 6))
    plt.pie(amounts, labels=categories, autopct='%1.1f%%')
    plt.title("Expenses by Category")
    plt.show()

r = tk.Tk()
r.title("Personal Finance Management System")
r.geometry("800x600")


n = ttk.Notebook(r)


t_income = ttk.Frame(n)
t_expenses = ttk.Frame(n)
t_savings = ttk.Frame(n)

n.add(t_income, text="Income")
n.add(t_expenses, text="Expenses")
n.add(t_savings, text="Savings")

n.pack(expand=1, fill='both')


label_income_source = tk.Label(t_income, text="Source")
entry_income_source = tk.Entry(t_income)
label_income_amount = tk.Label(t_income, text="Amount")
entry_income_amount = tk.Entry(t_income)
label_income_date = tk.Label(t_income, text="Date")
entry_income_date = tk.Entry(t_income)
button_add_income = tk.Button(t_income, text="Add Income", command=lambda: add_income(entry_income_source.get(), entry_income_amount.get(), entry_income_date.get()))

label_income_source.grid(row=0, column=0, padx=5, pady=5)
entry_income_source.grid(row=0, column=1, padx=5, pady=5)
label_income_amount.grid(row=1, column=0, padx=5, pady=5)
entry_income_amount.grid(row=1, column=1, padx=5, pady=5)
label_income_date.grid(row=2, column=0, padx=5, pady=5)
entry_income_date.grid(row=2, column=1, padx=5, pady=5)
button_add_income.grid(row=3, column=0, columnspan=2, pady=10)


income_tree = ttk.Treeview(t_income, columns=("ID", "Date", "Source", "Amount"), show='headings')
income_tree.heading("ID", text="ID")
income_tree.heading("Date", text="Date")
income_tree.heading("Source", text="Source")
income_tree.heading("Amount", text="Amount")
income_tree.grid(row=4, column=0, columnspan=2, pady=20)


label_expense_category = tk.Label(t_expenses, text="Category")
entry_expense_category = tk.Entry(t_expenses)
label_expense_amount = tk.Label(t_expenses, text="Amount")
entry_expense_amount = tk.Entry(t_expenses)
label_expense_date = tk.Label(t_expenses, text="Date")
entry_expense_date = tk.Entry(t_expenses)
button_add_expense = tk.Button(t_expenses, text="Add Expense", command=lambda: add_expense(entry_expense_category.get(), entry_expense_amount.get(), entry_expense_date.get()))

label_expense_category.grid(row=0, column=0, padx=5, pady=5)
entry_expense_category.grid(row=0, column=1, padx=5, pady=5)
label_expense_amount.grid(row=1, column=0, padx=5, pady=5)
entry_expense_amount.grid(row=1, column=1, padx=5, pady=5)
label_expense_date.grid(row=2, column=0, padx=5, pady=5)
entry_expense_date.grid(row=2, column=1, padx=5, pady=5)
button_add_expense.grid(row=3, column=0, columnspan=2, pady=10)



expense_tree = ttk.Treeview(t_expenses, columns=("ID", "Date", "Category", "Amount"), show='headings')
expense_tree.heading("ID", text="ID")
expense_tree.heading("Date", text="Date")
expense_tree.heading("Category", text="Category")
expense_tree.heading("Amount", text="Amount")
expense_tree.grid(row=4, column=0, columnspan=2, pady=20)


button_plot_expenses = tk.Button(t_expenses, text="Show Expense Pie Chart", command=plot_expense_pie_chart)
button_plot_expenses.grid(row=5, column=0, columnspan=2, pady=10)


label_savings_amount = tk.Label(t_savings, text="Amount")
entry_savings_amount = tk.Entry(t_savings)
label_savings_date = tk.Label(t_savings, text="Date")
entry_savings_date = tk.Entry(t_savings)
button_add_savings = tk.Button(t_savings, text="Add Savings", command=lambda: add_savings(entry_savings_amount.get(), entry_savings_date.get()))

label_savings_amount.grid(row=0, column=0, padx=5, pady=5)
entry_savings_amount.grid(row=0, column=1, padx=5, pady=5)
label_savings_date.grid(row=1, column=0, padx=5, pady=5)
entry_savings_date.grid(row=1, column=1, padx=5, pady=5)
button_add_savings.grid(row=2, column=0, columnspan=2, pady=10)


savings_tree = ttk.Treeview(t_savings, columns=("ID", "Date", "Amount"), show='headings')
savings_tree.heading("ID", text="ID")
savings_tree.heading("Date", text="Date")
savings_tree.heading("Amount", text="Amount")
savings_tree.grid(row=3, column=0, columnspan=2, pady=20)


setup_database()
refresh_income_list()
refresh_expense_list()
refresh_savings_list()

r.mainloop()
